In [1]:
import sys
sys.path.append('/home/jiajunb/neural-dimension-reduction')

In [2]:
import os

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import functional as F

from src.models.distance_modeling import SurveyorDataSet, Surveyor, thesis_kl_div_add_mse_loss, RetrieveSystem

import copy
from src.models.DenseNetwork import loss

torch.manual_seed(0)

In [3]:
device = torch.device('cuda:0')

In [4]:
best_model = Surveyor()
best_model.load_state_dict(torch.load('../saves/surveyor.on.multi-neg.sample.0.95238')['best_model'])
best_model.eval()

val_dataset = SurveyorDataSet.from_df('/home/jiajunb/neural-dimension-reduction/data/processed/sample/dev.csv')
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=1000, pin_memory=True)

gold = loss.nearest_neighbors(val_dataset.data.clone(), top_k=1, device='cuda')

In [5]:
best_model = best_model.to(device)

In [6]:
def extract_embeddings(data_loader, model):
    model.eval()
    embedding = list()
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            x = batch[0].to(device)
            out = model.encode_batch(x)
            embedding.append(out.cpu())
    return torch.cat(embedding, dim=0)

val_nn_loader = DataLoader(TensorDataset(val_dataset.data.clone()), batch_size=1000, pin_memory=True)
val_x_embedded = extract_embeddings(val_nn_loader, best_model)

In [7]:
retriever = RetrieveSystem(best_model)
block_list = torch.arange(val_x_embedded.shape[0])
cls_pred_nn_top, p_distances_nn_top = retriever.retrieve_corpus(val_x_embedded, block_list, val_x_embedded)


In [8]:
gold = loss.nearest_neighbors(val_dataset.data.clone(), top_k=20, device='cuda')

In [9]:
cls_pred_nn_top[:10, 0]

tensor([589, 589, 423, 589,  59, 589, 468,  84,  59,  59])

In [10]:
gold[1].view(-1)[:10]

tensor([413, 548, 433, 370, 461, 519,  84, 458, 462, 181])

In [11]:
(cls_pred_nn_top[:, 0] == gold[1].view(-1, 1)).sum()

tensor(10228)

In [12]:
len(gold[1])

666

In [13]:
# recall @1
float((cls_pred_nn_top[:, 0] == gold[1].view(-1)).sum()) / len(gold[1])

RuntimeError: The size of tensor a (666) must match the size of tensor b (13320) at non-singleton dimension 0

In [ ]:
cls_pred_nn_top[:10, :5]

In [ ]:
(cls_pred_nn_top[:, :5] == gold[1].reshape(-1, 1)).sum()

In [ ]:
# recall@5
float((cls_pred_nn_top[:, :5] == gold[1].view(-1, 1)).sum()) / len(gold[1])

In [ ]:
# recall@10
float((cls_pred_nn_top[:, :10] == gold[1]).sum()) / len(gold[1])

In [ ]:
# recall@20
float((cls_pred_nn_top[:, :20] == gold[1]).sum()) / len(gold[1])